In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import re
import pandas as pd

## 재무제표

### id와 ens 값 찾기

In [2]:
url_tmp = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930"

In [3]:
r_tmp = requests.get(url_tmp)

* encparam 및 id값에 대한 힌트 얻기

In [4]:
r_tmp.text.find("encparam:")

52164

In [5]:
r_tmp.text[52100:52300]

'30\',\r\n\t\t\t\t\t\tgrp_cd: grp==10?"01":"03",\r\n                        encparam: \'Ykk0ZXZrRFJuMVdWM084YURiVzE4dz09\'\r\n\t\t\t\t\t}\r\n\t\t\t\t, function (res) {\r\n\t\t\t\t\tvar data = []\r\n\t\t\t\t\t_.each(res.dt, function(v, i){\r\n\t'

In [6]:
r_tmp.text.find("id:")

66056

In [7]:
r_tmp.text[66000:66100]

": 'Ykk0ZXZrRFJuMVdWM084YURiVzE4dz09'\r\n                , id: 'QmZIZ20rMn' ? 'QmZIZ20rMn' : ''\r\n\t\t\t\t},"

* 어떤 형태인지 파악이 되었으니 정규식으로 값 뽑기

In [8]:
pattern_enc = re.compile("encparam: '(.+)'", re.IGNORECASE)
pattern_id  = re.compile("id: '(.+?)'", re.IGNORECASE)

In [9]:
target_text = r_tmp.text
encparam = pattern_enc.search(target_text).groups()[0]
id_ = pattern_id.search(target_text).groups()[0]

In [10]:
encparam

'Ykk0ZXZrRFJuMVdWM084YURiVzE4dz09'

In [11]:
id_

'QmZIZ20rMn'

* 가져온 정보를 바탕으로 전달 변수 생성

* freq_typ
----------
* A : 전체 
* Y : 연간 
* Q : 분기

* fin_type
---------
* 주재무제표 : 0
* K-IFRS(별도) : 3
* K-IFRS(연결) : 4
* K-GAAP(개별) : 1
* K-GAAP(연결) : 2

In [12]:
payload = {}
payload['cmp_cd'] = "005930"
payload['fin_typ'] = 4
payload['freq_typ']= 'Y'
payload['encparam'] = encparam
payload['id'] = id_

In [13]:
head = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/json',
    'Referer' : "https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?",
    'X-Requested-With': 'XMLHttpRequest'
}

In [14]:
payload

{'cmp_cd': '005930',
 'fin_typ': 4,
 'freq_typ': 'Y',
 'encparam': 'Ykk0ZXZrRFJuMVdWM084YURiVzE4dz09',
 'id': 'QmZIZ20rMn'}

In [15]:
finacial_url = "https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?"

In [16]:
r = requests.get(finacial_url, params=payload, headers = head)

In [17]:
r.text

'<table class="hQmZIZ20rMn gHead01 all-width" summary="주요재무정보를 제공합니다."><caption class="blind">"주요재무정보"</caption><thead><tr><th>6958</th><th>6958</th><th>6958</th><th>6958</th><th>6958</th><th>6958</th><th>6958</th><th>6958</th><th>6958</th></tr></thead><tbody><tr><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td></tr><tr><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td></tr><tr><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td></tr><tr><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td></tr><tr><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td></tr><tr><td>6,958</td><td>6,958</td><td>6,958</td><td>6,958</td><td>6

In [18]:
financial = pd.read_html(r.text)[1]

In [19]:
financial

주요재무정보                연간                                      \
        주요재무정보 2014/12  (IFRS연결) 2015/12  (IFRS연결) 2016/12  (IFRS연결)   
0          매출액      2.062060e+06      2.006535e+06      2.018667e+06   
1         영업이익      2.502510e+05      2.641340e+05      2.924070e+05   
2   영업이익(발표기준)      2.502510e+05      2.641340e+05      2.924070e+05   
3     세전계속사업이익      2.787500e+05      2.596100e+05      3.071370e+05   
4        당기순이익      2.339440e+05      1.906010e+05      2.272610e+05   
5    당기순이익(지배)      2.308250e+05      1.869460e+05      2.241570e+05   
6   당기순이익(비지배)      3.119000e+03      3.655000e+03      3.104000e+03   
7         자산총계      2.304230e+06      2.421795e+06      2.621743e+06   
8         부채총계      6.233480e+05      6.311970e+05      6.921130e+05   
9         자본총계      1.680882e+06      1.790598e+06      1.929630e+06   
10    자본총계(지배)      1.621817e+06      1.728768e+06      1.864243e+06   
11   자본총계(비지배)      5.906500e+04      6.183000e+04      6.538700e+04   
12         자본금      8.975000e+03      8.975000e+03      8.975000e+03   
13    영업활동현금흐름      3.697540e+05      4.006180e+05      4.738560e+05   
14    투자활동현금흐름     -3.280640e+05     -2.716780e+05     -2.965870e+05   
15    재무활동현금흐름     -3.057100e+04     -6.573500e+04     -8.669500e+04   
16       CAPEX      2.204290e+05      2.588020e+05      2.414300e+05   
17         FCF      1.493240e+05      1.418150e+05      2.324270e+05   
18      이자발생부채      1.126550e+05      1.287400e+05      1.528240e+05   
19       영업이익률      1.214000e+01      1.316000e+01      1.449000e+01   
20        순이익률      1.134000e+01      9.500000e+00      1.126000e+01   
21      ROE(%)      1.506000e+01      1.116000e+01      1.248000e+01   
22      ROA(%)      1.053000e+01      8.070000e+00      9.010000e+00   
23        부채비율      3.709000e+01      3.525000e+01      3.587000e+01   
24       자본유보율      1.937947e+04      2.111788e+04      2.200414e+04   
25      EPS(원)      2.713000e+03      2.198000e+03      2.735000e+03   
26      PER(배)      9.780000e+00      1.147000e+01      1.318000e+01   
27      BPS(원)      2.166400e+04      2.371500e+04      2.663600e+04   
28      PBR(배)      1.230000e+00      1.060000e+00      1.350000e+00   
29    현금DPS(원)      4.000000e+02      4.200000e+02      5.700000e+02   
30     현금배당수익률      1.510000e+00      1.670000e+00      1.580000e+00   
31   현금배당성향(%)      1.300000e+01      1.642000e+01      1.781000e+01   
32  발행주식수(보통주)      7.364967e+09      7.364967e+09      7.033967e+09   

                                                             \
   2017/12  (IFRS연결) 2018/12  (IFRS연결) 2019/12(E)  (IFRS연결)   
0       2.395754e+06      2.437714e+06           2265682.00   
1       5.364500e+05      5.888670e+05            267450.00   
2       5.364500e+05      5.888670e+05                  NaN   
3       5.619600e+05      6.116000e+05            297285.00   
4       4.218670e+05      4.434490e+05            217736.00   
5       4.134460e+05      4.389090e+05            214016.00   
6       8.422000e+03               NaN                  NaN   
7       3.017521e+06      3.393572e+06           3535995.00   
8       8.726070e+05      9.160410e+05            926377.00   
9       2.144914e+06      2.477532e+06           2604981.00   
10      2.072134e+06      2.400690e+06           2524513.00   
11      7.278000e+04               NaN                  NaN   
12      8.975000e+03      8.975000e+03              8979.00   
13      6.216200e+05      6.703190e+05            449643.00   
14     -4.938520e+05     -5.224050e+05           -299868.00   
15     -1.256090e+05     -1.509020e+05           -107841.00   
16      4.279220e+05      2.955640e+05            261414.00   
17      1.936980e+05      3.747550e+05            185759.00   
18      1.881400e+05      1.466710e+05                  NaN   
19      2.239000e+01      2.416000e+01                11.80   
20      1.761000e+01      1.819000e+01                 9.61   
21      2.101000e+01      1.963000e+01

In [20]:
financial.transpose()

0       1           2         3   \
주요재무정보 주요재무정보                        매출액    영업이익  영업이익(발표기준)  세전계속사업이익   
연간     2014/12  (IFRS연결)     2.06206e+06  250251      250251    278750   
       2015/12  (IFRS연결)     2.00654e+06  264134      264134    259610   
       2016/12  (IFRS연결)     2.01867e+06  292407      292407    307137   
       2017/12  (IFRS연결)     2.39575e+06  536450      536450    561960   
       2018/12  (IFRS연결)     2.43771e+06  588867      588867    611600   
       2019/12(E)  (IFRS연결)  2.26568e+06  267450         NaN    297285   
       2020/12(E)  (IFRS연결)  2.41561e+06  348911         NaN    381569   
       2021/12(E)  (IFRS연결)  2.54745e+06  431508         NaN    467387   

                                 4          5           6            7   \
주요재무정보 주요재무정보                 당기순이익  당기순이익(지배)  당기순이익(비지배)         자산총계   
연간     2014/12  (IFRS연결)     233944     230825        3119  2.30423e+06   
       2015/12  (IFRS연결)     190601     186946        3655   2.4218e+06   
       2016/12  (IFRS연결)     227261     224157        3104  2.62174e+06   
       2017/12  (IFRS연결)     421867     413446        8422  3.01752e+06   
       2018/12  (IFRS연결)     443449     438909         NaN  3.39357e+06   
       2019/12(E)  (IFRS연결)  217736     214016         NaN    3.536e+06   
       2020/12(E)  (IFRS연결)  279155     273442         NaN  3.73683e+06   
       2021/12(E)  (IFRS연결)  342270     338938         NaN   4.0046e+06   

                                 8            9   ...     23       24      25  \
주요재무정보 주요재무정보                  부채총계         자본총계  ...   부채비율    자본유보율  EPS(원)   
연간     2014/12  (IFRS연결)     623348  1.68088e+06  ...  37.09  19379.5    2713   
       2015/12  (IFRS연결)     631197   1.7906e+06  ...  35.25  21117.9    2198   
       2016/12  (IFRS연결)     692113  1.92963e+06  ...  35.87  22004.1    2735   
       2017/12  (IFRS연결)     872607  2.14491e+06  ...  40.68  24536.1    5421   
       2018/12  (IFRS연결)     916041  2.47753e+06  ...  36.97  27531.9    6024   
       2019/12(E)  (IFRS연결)  926377  2.60498e+06  ...  35.56      NaN    3151   
       2020/12(E)  (IFRS연결)  959697  2.77256e+06  ...  34.61      NaN    4026   
       2021/12(E)  (IFRS연결)  985080  3.01425e+06  ...  32.68      NaN    4990   

                                 26      27      28        29       30  \
주요재무정보 주요재무정보                PER(배)  BPS(원)  PBR(배)  현금DPS(원)  현금배당수익률   
연간     2014/12  (IFRS연결)       9.78   21664    1.23       400     1.51   
       2015/12  (IFRS연결)      11.47   23715    1.06       420     1.67   
       2016/12  (IFRS연결)      13.18   26636    1.35       570     1.58   
       2017/12  (IFRS연결)        9.4   30427    1.67       850     1.67   
       2018/12  (IFRS연결)       6.42   35342    1.09      1416     3.66   
       2019/12(E)  (IFRS연결)   14.85   37165    1.26      1440     3.08   
       2020/12(E)  (IFRS연결)   11.63   39559    1.18      1614     3.45   
       2021/12(E)  (IFRS연결)    9.38   43129    1.09      1612     3.44   

                                    31           32  
주요재무정보 주요재무정보                현금배당성향(%)   발행주식수(보통주)  
연간     2014/12  (IFRS연결)            13  7.36497e+09  
       2015/12  (IFRS연결)         16.42  7.36497e+09  
       2016/12  (IFRS연결)         17.81  7.03397e+09  
       2017/12  (IFRS연결)         14.09  6.45492e+09  
       2018/12  (IFRS연결)         21.92  5.96978e+09  
       2019/12(E)  (IFRS연결)      40.16          NaN  
       2020/12(E)  (IFRS연결)      35.23          NaN  
       2021/12(E)  (IFRS연결)      28.39          NaN  

[9 rows x 33 columns]

In [21]:
financial.transpose().reset_index()

,level_0,level_1,0,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,32
0,주요재무정보,주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
1,연간,2014/12 (IFRS연결),2.06206e+06,250251,250251,278750,233944,230825,3119,2.30423e+06,...,37.09,19379.5,2713,9.78,21664,1.23,400,1.51,13,7.36497e+09
2,연간,2015/12 (IFRS연결),2.00654e+06,264134,264134,259610,190601,186946,3655,2.4218e+06,...,35.25,21117.9,2198,11.47,23715,1.06,420,1.67,16.42,7.36497e+09
3,연간,2016/12 (IFRS연결),2.01867e+06,292407,292407,307137,227261,224157,3104,2.62174e+06,...,35.87,22004.1,2735,13.18,26636,1.35,570,1.58,17.81,7.03397e+09
4,연간,2017/12 (IFRS연결),2.39575e+06,536450,536450,561960,421867,413446,8422,3.01752e+06,...,40.68,24536.1,5421,9.4,30427,1.67,850,1.67,14.09,6.45492e+09
5,연간,2018/12 (IFRS연결),2.43771e+06,588867,588867,611600,443449,438909,NaN,3.39357e+06,...,36.97,27531.9,6024,6.42,35342,1.09,1416,3.66,21.92,5.96978e+09
6,연간,2019/12(E) (IFRS연결),2.26568e+06,267450,NaN,297285,217736,214016,NaN,3.536e+06,...,35.56,NaN,3151,14.85,37165,1.26,1440,3.08,40.16,NaN
7,연간,2020/12(E) (IFRS연결),2.41561e+06,348911,NaN,381569,279155,273442,NaN,3.73683e+06,...,34.61,NaN,4026,11.63,39559,1.18,1614,3.45,35.23,NaN
8,연간,2021/12(E) (IFRS연결),2.54745e+06,431508,NaN,467387,342270,338938,NaN,4.0046e+06,...,32.68,NaN,4990,9.38,43129,1.09,1612,3.44,28.39,NaN


In [22]:
finalcial_trans = financial.transpose().reset_index()

In [23]:
finalcial_trans

,level_0,level_1,0,1,2,3,4,5,6,7,...,23,24,25,26,27,28,29,30,31,32
0,주요재무정보,주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
1,연간,2014/12 (IFRS연결),2.06206e+06,250251,250251,278750,233944,230825,3119,2.30423e+06,...,37.09,19379.5,2713,9.78,21664,1.23,400,1.51,13,7.36497e+09
2,연간,2015/12 (IFRS연결),2.00654e+06,264134,264134,259610,190601,186946,3655,2.4218e+06,...,35.25,21117.9,2198,11.47,23715,1.06,420,1.67,16.42,7.36497e+09
3,연간,2016/12 (IFRS연결),2.01867e+06,292407,292407,307137,227261,224157,3104,2.62174e+06,...,35.87,22004.1,2735,13.18,26636,1.35,570,1.58,17.81,7.03397e+09
4,연간,2017/12 (IFRS연결),2.39575e+06,536450,536450,561960,421867,413446,8422,3.01752e+06,...,40.68,24536.1,5421,9.4,30427,1.67,850,1.67,14.09,6.45492e+09
5,연간,2018/12 (IFRS연결),2.43771e+06,588867,588867,611600,443449,438909,NaN,3.39357e+06,...,36.97,27531.9,6024,6.42,35342,1.09,1416,3.66,21.92,5.96978e+09
6,연간,2019/12(E) (IFRS연결),2.26568e+06,267450,NaN,297285,217736,214016,NaN,3.536e+06,...,35.56,NaN,3151,14.85,37165,1.26,1440,3.08,40.16,NaN
7,연간,2020/12(E) (IFRS연결),2.41561e+06,348911,NaN,381569,279155,273442,NaN,3.73683e+06,...,34.61,NaN,4026,11.63,39559,1.18,1614,3.45,35.23,NaN
8,연간,2021/12(E) (IFRS연결),2.54745e+06,431508,NaN,467387,342270,338938,NaN,4.0046e+06,...,32.68,NaN,4990,9.38,43129,1.09,1612,3.44,28.39,NaN


In [24]:
finalcial_trans.columns = finalcial_trans.loc[0]

In [25]:
finalcial_trans

,주요재무정보,주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
0,주요재무정보,주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
1,연간,2014/12 (IFRS연결),2.06206e+06,250251,250251,278750,233944,230825,3119,2.30423e+06,...,37.09,19379.5,2713,9.78,21664,1.23,400,1.51,13,7.36497e+09
2,연간,2015/12 (IFRS연결),2.00654e+06,264134,264134,259610,190601,186946,3655,2.4218e+06,...,35.25,21117.9,2198,11.47,23715,1.06,420,1.67,16.42,7.36497e+09
3,연간,2016/12 (IFRS연결),2.01867e+06,292407,292407,307137,227261,224157,3104,2.62174e+06,...,35.87,22004.1,2735,13.18,26636,1.35,570,1.58,17.81,7.03397e+09
4,연간,2017/12 (IFRS연결),2.39575e+06,536450,536450,561960,421867,413446,8422,3.01752e+06,...,40.68,24536.1,5421,9.4,30427,1.67,850,1.67,14.09,6.45492e+09
5,연간,2018/12 (IFRS연결),2.43771e+06,588867,588867,611600,443449,438909,NaN,3.39357e+06,...,36.97,27531.9,6024,6.42,35342,1.09,1416,3.66,21.92,5.96978e+09
6,연간,2019/12(E) (IFRS연결),2.26568e+06,267450,NaN,297285,217736,214016,NaN,3.536e+06,...,35.56,NaN,3151,14.85,37165,1.26,1440,3.08,40.16,NaN
7,연간,2020/12(E) (IFRS연결),2.41561e+06,348911,NaN,381569,279155,273442,NaN,3.73683e+06,...,34.61,NaN,4026,11.63,39559,1.18,1614,3.45,35.23,NaN
8,연간,2021/12(E) (IFRS연결),2.54745e+06,431508,NaN,467387,342270,338938,NaN,4.0046e+06,...,32.68,NaN,4990,9.38,43129,1.09,1612,3.44,28.39,NaN


In [28]:
finalcial_trans.drop([0],inplace=True)

In [29]:
finalcial_trans

,주요재무정보,주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
1,연간,2014/12 (IFRS연결),2.06206e+06,250251,250251,278750,233944,230825,3119,2.30423e+06,...,37.09,19379.5,2713,9.78,21664,1.23,400,1.51,13,7.36497e+09
2,연간,2015/12 (IFRS연결),2.00654e+06,264134,264134,259610,190601,186946,3655,2.4218e+06,...,35.25,21117.9,2198,11.47,23715,1.06,420,1.67,16.42,7.36497e+09
3,연간,2016/12 (IFRS연결),2.01867e+06,292407,292407,307137,227261,224157,3104,2.62174e+06,...,35.87,22004.1,2735,13.18,26636,1.35,570,1.58,17.81,7.03397e+09
4,연간,2017/12 (IFRS연결),2.39575e+06,536450,536450,561960,421867,413446,8422,3.01752e+06,...,40.68,24536.1,5421,9.4,30427,1.67,850,1.67,14.09,6.45492e+09
5,연간,2018/12 (IFRS연결),2.43771e+06,588867,588867,611600,443449,438909,NaN,3.39357e+06,...,36.97,27531.9,6024,6.42,35342,1.09,1416,3.66,21.92,5.96978e+09
6,연간,2019/12(E) (IFRS연결),2.26568e+06,267450,NaN,297285,217736,214016,NaN,3.536e+06,...,35.56,NaN,3151,14.85,37165,1.26,1440,3.08,40.16,NaN
7,연간,2020/12(E) (IFRS연결),2.41561e+06,348911,NaN,381569,279155,273442,NaN,3.73683e+06,...,34.61,NaN,4026,11.63,39559,1.18,1614,3.45,35.23,NaN
8,연간,2021/12(E) (IFRS연결),2.54745e+06,431508,NaN,467387,342270,338938,NaN,4.0046e+06,...,32.68,NaN,4990,9.38,43129,1.09,1612,3.44,28.39,NaN


In [ ]:
finalcial_trans['종목코드'] = '005930'

In [ ]:
finalcial_trans